In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [2]:
import sys
import os

if not ".." in sys.path:
    sys.path.append("..")

In [3]:
from lit_modules.litdatamodules.lit_imagenet import LitImageNetDataModule
from lit_modules.litdatamodules.lit_lamem import LitLaMemDataModule

import lightning as L
from lightning.pytorch.utilities.types import TRAIN_DATALOADERS
from lightning.pytorch.utilities import CombinedLoader
from lightning.pytorch.callbacks import (
    DeviceStatsMonitor,
    StochasticWeightAveraging,
    LearningRateMonitor,
)

from torchvision import transforms
from torch.utils.data import DataLoader

from typing import Dict, List
import pandas as pd

from lit_modules import LitResNet50
from lightning import Trainer

In [4]:
class LitCombineDataModule(L.LightningDataModule):
    def __init__(
        self,
        root_imagenet: str,
        meta_path_imagenet: str,
        root_lamem: str,
        num_workers: int = 10,
        batch_size: int = 32,
        mode: str = "min_size",
        desired_image_size: int = 224,
    ) -> None:
        super().__init__()
        self.save_hyperparameters("batch_size")
        self.save_hyperparameters("desired_image_size")
        self.mode = mode

        self.data_modules = {
            "regression": LitLaMemDataModule(
                root=root_lamem,
                num_workers=num_workers,
                batch_size=batch_size,
                desired_image_size=desired_image_size,
            ),
            "classification": LitImageNetDataModule(
                root=root_imagenet,
                meta_path=meta_path_imagenet,
                num_workers=num_workers,
                batch_size=batch_size,
                desired_image_size=desired_image_size,
            ),
        }

    def setup(self, stage: str) -> None:

        for key, value in self.data_modules.items():
            value.setup(stage)

    def train_dataloader(self):
        data_loaders = {
            "regression": self.data_modules["regression"].train_dataloader(),
            "classification": self.data_modules["classification"].train_dataloader(),
        }

        return CombinedLoader(data_loaders, mode=self.mode)

    def val_dataloader(self):
        data_loaders = {
            "regression": self.data_modules["regression"].val_dataloader(),
            "classification": self.data_modules["classification"].val_dataloader(),
        }

        return CombinedLoader(data_loaders, mode=self.mode)

In [5]:
root_imagenet = "/datashare/ImageNet/ILSVRC2012"
meta_path_imagenet = (
    "/home/soroush1/projects/def-kohitij/soroush1/pretrain-imagenet/data/ImageNet"
)
root_lamem = "/home/soroush1/projects/def-kohitij/soroush1/pretrain-imagenet/data/lamem/lamem_images/lamem"
batch_size = 1
ds = LitCombineDataModule(
    root_imagenet=root_imagenet,
    meta_path_imagenet=meta_path_imagenet,
    root_lamem=root_lamem,
    batch_size=64,
    mode="min_size",
    num_workers=2,
)

ds.setup("TRAIN")

train_loader = ds.train_dataloader()

for i, (batch, batch_idx, dataloader_idx) in enumerate(train_loader):
    print(f"{batch.keys()}, {batch_idx=}, {dataloader_idx=}")

    if i == 2:
        break

num_output_features = 1000
model = LitResNet50(
    learning_rate=1e-3,
    num_output_features=num_output_features,
    task="classification",
)

lr_monitor = LearningRateMonitor(logging_interval="epoch")

trainer = Trainer(
    max_epochs=200,
    # callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=10)],
    # callbacks=[DeviceStatsMonitor(), lr_monitor],
    callbacks=[lr_monitor],
    fast_dev_run=False,
    # devices=[0, 1, 2, 3],
    accelerator="gpu",
    num_nodes=1,
    # strategy=strategy,
    overfit_batches=1,
    # gradient_clip_val=0.5,
    # gradient_clip_algorithm="value",
    # logger=tb_logger,
)

trainer.fit(
    model=model,
    datamodule=ds,
)

dict_keys(['regression', 'classification']), batch_idx=0, dataloader_idx=0
dict_keys(['regression', 'classification']), batch_idx=1, dataloader_idx=0
dict_keys(['regression', 'classification']), batch_idx=2, dataloader_idx=0


/home/soroush1/projects/def-kohitij/soroush1/pretrain-imagenet/.venv/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/soroush1/projects/def-kohitij/soroush1/pretrai ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.
/home/soroush1/projects/def-kohitij/soroush1/pretrain-imagenet/.venv/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/soroush1/projects/def-kohitij/soroush1/pretrai ...
2024-03

x.size() = torch.Size([64, 1000])


Sanity Checking: |                                                                                            …

key = 'regression'
x.size() = torch.Size([64, 1000])
validation: output.size() = torch.Size([64, 1])
key = 'classification'
validation: output.size() = torch.Size([64, 1000])


...

key = 'regression'
x.size() = torch.Size([64, 1000])
validation: output.size() = torch.Size([64, 1])
key = 'classification'
validation: output.size() = torch.Size([64, 1000])
x.size() = torch.Size([64, 1000])
output.size() = torch.Size([64, 1])
output.size() = torch.Size([64, 1000])


Validation: |                                                                                                 …

key = 'regression'
x.size() = torch.Size([64, 1000])


`Trainer.fit` stopped: `max_epochs=200` reached.


validation: output.size() = torch.Size([64, 1])
key = 'classification'
validation: output.size() = torch.Size([64, 1000])


# Lalem Data Preprocessing

In [ ]:
# split_path = "/home/soroush1/projects/def-kohitij/soroush1/pretrain-imagenet/data/lamem/lamem_images/lamem/splits/"
# splits = list(filter(lambda x: ".txt" in x, os.listdir(split_path)))

# for split in splits:
#     results = {"image_name": [], "memo_score": []}
#     with open(os.path.join(split_path, split), "r") as f:
#         for x in f:
#             image_name, mem_score = x.split(" ")
#             results["image_name"].append(image_name.strip())
#             results["memo_score"].append(float(mem_score.strip()))

#     file_name, _ = split.split(".")
#     pd.DataFrame(results).to_csv(
#         os.path.join(split_path, f"{file_name}.csv"), index=False
#     )